In [61]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from lightgbm import LGBMRegressor
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 1000000

In [3]:
df = pd.read_csv('merge.csv')
df = df[df['trace_category']!='noise']

C:\Users\Zhafran\.conda\envs\ExVodka\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (7,11,13,14,15,18,19,20,21,22,24,25,26,30,31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df.head()

,network_code,receiver_code,receiver_type,receiver_latitude,receiver_longitude,receiver_elevation_m,p_arrival_sample,p_status,p_weight,p_travel_sec,...,source_magnitude_author,source_mechanism_strike_dip_rake,source_distance_deg,source_distance_km,back_azimuth_deg,snr_db,coda_end_sample,trace_start_time,trace_category,trace_name
235426,TA,109C,BH,32.8889,-117.1051,150.0,700.0,manual,0.5,17.080000,...,None,None,0.92,102.09,159.3,[56.79999924 55.40000153 47.40000153],[[2896.]],2006-07-23 15:59:00.960000,earthquake_local,109C.TA_20060723155859_EV
235427,TA,109C,BH,32.8889,-117.1051,150.0,600.0,manual,0.5,16.879999,...,None,None,0.91,101.34,281.7,[65. 65.5 61.40000153],[[5508.]],2006-11-03 15:56:53.610000,earthquake_local,109C.TA_20061103155652_EV
235428,TA,109C,BH,32.8889,-117.1051,150.0,500.0,manual,0.5,17.260000,...,None,None,0.92,101.87,280.5,[37.20000076 42. 38.59999847],[[3114.]],2006-11-03 16:12:24.700000,earthquake_local,109C.TA_20061103161223_EV
235429,TA,109C,BH,32.8889,-117.1051,150.0,900.0,manual,0.5,17.280001,...,None,None,0.93,103.26,281.6,[54.09999847 54.90000153 45.5 ],[[3152.]],2006-11-14 13:32:22.540000,earthquake_local,109C.TA_20061114133221_EV
235430,TA,109C,BH,32.8889,-117.1051,150.0,700.0,manual,0.5,18.139999,...,None,None,0.92,102.48,4.7,[58.20000076 56.20000076 53.79999924],[[3134.]],2006-11-27 10:46:41.060000,earthquake_local,109C.TA_20061127104640_EV


In [109]:
df.iloc[2]

network_code                                                           TA
receiver_code                                                        109C
receiver_type                                                          BH
receiver_latitude                                                 32.8889
receiver_longitude                                              -117.1051
receiver_elevation_m                                                150.0
p_arrival_sample                                                    500.0
p_status                                                           manual
p_weight                                                              0.5
p_travel_sec                                                        17.26
s_arrival_sample                                                   1678.0
s_status                                                           manual
s_weight                                                              0.5
source_id                             

In [205]:
from sklearn.metrics import mean_squared_error

def calculate_epicenter():
    error = []
    for mult in np.arange(7, 9, 0.1):
        df['delta_t_second'] = (df['s_arrival_sample'] - df['p_arrival_sample'])/100
        df['pred_distance'] = df['delta_t_second']*mult
        mse = mean_squared_error(df['source_distance_km'], df['pred_distance'])
        rmse = mse**0.5
        error.append([rmse, mult])
    return np.array(error).round(4)

In [206]:
p = calculate_epicenter()
p

array([[13.8492,  7.    ],
       [13.3881,  7.1   ],
       [12.971 ,  7.2   ],
       [12.6022,  7.3   ],
       [12.2861,  7.4   ],
       [12.0269,  7.5   ],
       [11.8282,  7.6   ],
       [11.6932,  7.7   ],
       [11.6242,  7.8   ],
       [11.6221,  7.9   ],
       [11.6872,  8.    ],
       [11.8183,  8.1   ],
       [12.0132,  8.2   ],
       [12.269 ,  8.3   ],
       [12.5818,  8.4   ],
       [12.9475,  8.5   ],
       [13.3619,  8.6   ],
       [13.8204,  8.7   ],
       [14.319 ,  8.8   ],
       [14.8535,  8.9   ]])

In [207]:
df['delta_t_second'] = (df['s_arrival_sample'] - df['p_arrival_sample'])/100
df['pred_distance'] = df['delta_t_second']*7.9

In [210]:
df['pred_distance'][500:510]

235926    37.288
235927    67.703
235928    36.182
235929    69.283
235930    30.415
235931    34.128
235932    27.808
235933    52.140
235934    27.492
235935    28.835
Name: pred_distance, dtype: float64

In [211]:
df['source_distance_km'][500:510]

235926    21.93
235927    58.57
235928    25.68
235929    74.67
235930    19.22
235931    26.58
235932    17.49
235933    32.07
235934    28.35
235935    28.65
Name: source_distance_km, dtype: float64